In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import glob

In [2]:
# setting visible columns to max
pd.set_option('display.max_columns', None)

In [3]:
def create_price_history_dataframe(path):

    # Creating a list of all csv files in path
    csv_files = glob.glob(path + '*.csv')

    # empty list to store all dataframes
    dataframes = list()
    
    # Droping NIRAJSPAT as it contains no data
    csv_files.remove('Stock History/NIRAJISPAT.NS.csv')

    # using for loop to itterate thouigh all the files in the path and make a single dataframe
    for file in csv_files:
        #print(f'Formating {file}')
        
        frame = pd.read_csv(file)
        
        # Adding file name to column of dataframe 
        frame['Ticker'] = file[14:-7]
        
        dataframes.append(frame)
           
    return dataframes

In [4]:
def protfolio_calculator(df):
    
    # Using the opening and closing price to calculate daily chainge in price
    df['Change'] = (df.Close - df.Open)/df.Open
    
    # Calculating stock unit's held on the opening of first day (assuming Rs 10,000 intital capital)
    df.loc[0,'Units Held'] = (10000/df.iloc[0].Open)

    # Calculating units held as end of each day, to be used for accounting for stock splits and dividened yeild
    for i in range(1,len(df)):
        if df.loc[i,'Stock Splits'] > 0:
            df.loc[i,'Units Held'] = df.loc[i-1,'Units Held']*df.loc[i,'Stock Splits']
        else: df.loc[i,'Units Held'] = df.loc[i-1,'Units Held']

    # Calculating Capital Gains from dividends
    df['Dividend Gains'] = df['Units Held'] * df['Dividends']
 
    # Calculating Opening and Closing Balance each day
    df['Opening Balance'] = (df['Units Held'] * df['Open'])

    df['Closing Balance'] = (df['Units Held'] * df['Close']) + df['Dividend Gains']
    
    return df

In [5]:
def get_stock_history():
    
    path = 'Stock History/'
    history = list()

    dataframes = create_price_history_dataframe(path)

    for frame in dataframes:
        history.append(protfolio_calculator(frame))

    return pd.concat(history)

In [6]:
df = get_stock_history()

In [7]:
stockHistory = df.copy(deep=True)

In [8]:
stockHistory.set_index(['Ticker','Date'],inplace=True)

In [9]:
stockHistory.head()

Open        High         Low       Close  \
Ticker     Date                                                         
DATAPATTNS 2021-12-24  856.049988  864.000000  743.849976  755.000000   
           2021-12-27  738.500000  741.700012  702.349976  705.750000   
           2021-12-28  705.750000  748.000000  702.000000  734.599976   
           2021-12-29  748.000000  824.000000  740.200012  787.250000   
           2021-12-30  792.450012  814.000000  772.049988  778.299988   

                           Volume  Dividends  Stock Splits    Change  \
Ticker     Date                                                        
DATAPATTNS 2021-12-24  13492548.0        0.0           0.0 -0.118042   
           2021-12-27   2137929.0        0.0           0.0 -0.044347   
           2021-12-28   1586404.0        0.0           0.0  0.040878   
           2021-12-29   5408945.0        0.0           0.0  0.052473   
           2021-12-30   1904086.0        0.0           0.0 -0.017856   

                       Units Held  Dividend Gains  Opening Balance  \
Ticker     Date                                                      
DATAPATTNS 2021-12-24   11.681561             0.0     10000.000000   
           2021-12-27   11.681561             0.0      8626.832668   
           2021-12-28   11.681561             0.0      8244.261551   
           2021-12-29   11.681561             0.0      8737.807496   
           2021-12-30   11.681561             0.0      9257.053017   

                       Closing Balance  
Ticker     Date                         
DATAPATTNS 2021-12-24      8819.578421  
           2021-12-27      8244.261551  
           2021-12-28      8581.274295  
           2021-12-29      9196.308758  
           2021-12-30      9091.758646

In [10]:
stockHistory.loc['DATAPATTNS']

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Change,Units Held,Dividend Gains,Opening Balance,Closing Balance
Date,,,,,,,,,,,,
2021-12-24,856.049988,864.000000,743.849976,755.000000,13492548.0,0.0,0.0,-0.118042,11.681561,0.0,10000.000000,8819.578421
2021-12-27,738.500000,741.700012,702.349976,705.750000,2137929.0,0.0,0.0,-0.044347,11.681561,0.0,8626.832668,8244.261551
2021-12-28,705.750000,748.000000,702.000000,734.599976,1586404.0,0.0,0.0,0.040878,11.681561,0.0,8244.261551,8581.274295
2021-12-29,748.000000,824.000000,740.200012,787.250000,5408945.0,0.0,0.0,0.052473,11.681561,0.0,8737.807496,9196.308758
2021-12-30,792.450012,814.000000,772.049988,778.299988,1904086.0,0.0,0.0,-0.017856,11.681561,0.0,9257.053017,9091.758646
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-11,833.400024,852.799988,824.950012,832.200012,114922.0,0.0,0.0,-0.001440,11.681561,0.0,9735.413075,9721.395060
2022-08-12,832.099976,843.700012,821.049988,838.549988,104061.0,0.0,0.0,0.007751,11.681561,0.0,9720.226476,9795.572686
2022-08-16,840.000000,862.000000,828.900024,859.799988,159346.0,0.0,0.0,0.023571,11.681561,0.0,9812.511091,10043.805853
